In [29]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sklearn

import os
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

In [152]:
# need to figure out how to use the environment variables #

client_id = "a746fc7abc444aec9750efd4ec4cd709"
client_secret = "1b1fe2c07b5b4044a184709d94c80a27"
redirect_uri='https://localhost/'
username = "dj_daboss"
scope = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [153]:
results = sp.user_playlists('dj_daboss')
valid = set(['DNB','RAP','Dub','Cheeky Cheekster','La Casaaa'])

playlist_ids = []
playlist_names = []

for l in results['items']:
    name = l['name']
    if name in valid:
        playlist_names.append(l['name'])
        playlist_ids.append(l['id'])

print(playlist_names)
print(playlist_ids)

['Dub', 'La Casaaa', 'DNB', 'Cheeky Cheekster', 'RAP']
['4wSbtr18J595mkWMfD928m', '0XL66SfqEC2Tl2ijSbGIcu', '4FESBW9zpPjTIaiqYYbwK5', '6i8494qewrd4m6eu3xNtYD', '6ovmjxF6JsA9SIvllYZg6C']


In [154]:
def timbreAnalysis(id):
    audio_analysis = sp.audio_analysis(id)
    sections = audio_analysis['sections']
    segments = audio_analysis['segments']
    timbres = []

    for i,sec in enumerate(sections):   
        section_start = sections[i]['start']
        section_end = section_start+sections[i]['duration']

        section_segments = []
        for seg in segments:
            if seg['start'] > section_start and seg['start'] < section_end:
                section_segments.append(seg)

        section_timbres = []
        for seg in section_segments:
            section_timbres.append(np.array(seg['timbre']))

        timbres.append(np.mean(section_timbres, axis=0))
    return np.mean(timbres, axis=0)

https://github.com/spotify/web-api/issues/947

In [157]:
def get_audio_features(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id)
    songs = results['items']
    
    # results limited to 100 tracks #
    while results['next']:
        results = sp.next(results)
        songs.extend(results['items'])
    
    ids = []
    names = []
        
    for i in range(len(songs)):
        ids.append(songs[i]['track']['id'])
        names.append(songs[i]['track']['name'])
    
    ids = list(filter(None, ids))
    
    features = []
    
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        
        for track in audio_features:
            features.append(track)

    features = list(filter(None, features))
    df = pd.DataFrame(features)
    df['name'] = names
    df.drop(['type','uri','track_href','analysis_url'], inplace=True,axis=1)

    return df


In [ ]:
def get_audio_analysis(song_ids):
    for i in range(len(song_ids)):
        
        


In [160]:
irrelevant_ftrs = ['id', 'duration_ms', 'name', 'mode', 'liveness', 'key']

def get_objects(username, playlist_id, irrelevant_ftrs):
    df = get_features(username, playlist_id, False)
    df_ftrs = pd.DataFrame(df.drop(irrelevant_ftrs, axis=1))
    song_names = set()
    for name in df['name']:
        song_names.add(name)
    
    return df, df_ftrs, song_names

dubstep_df, dubstep_ftrs, dubstep_songs = get_objects(username, playlist_ids[0], irrelevant_ftrs)
# house_df, house_ftrs, house_songs = get_objects(username, playlist_ids[1], irrelevant_ftrs)
# dnb_df, dnb_ftrs, dnb_songs = get_objects(username, playlist_ids[2], irrelevant_ftrs)


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5EhnuLMer8qAXEDwmLtyrW,6OTClxme7EYRZGO6An3SMc,6Ep94gYKvkI4OqCOGV66Wx,6uwjDA0Qi6hk8C6lPJIMc9,2il9NDpW87Hk2L9AjeDf8r,0zOqoAh2thAbmju80p850h,0fy3MozJBrM858gEIo3Yec,0mE1Eo3encgJEIPzSLKTSb,6GXB5IyqwhZMdyYqpPkW1X,3tCJOzicyEdN3CzbLvKVxZ,71LZ7lwCisTG3H9UXKraTe,2pMbqyksGFuvLWkAdHRE4V,5nz3dwffS8W2WDWRa9SgfR,5uYcKM87XNEfeT6rxrTQOs,4JHL5XOuByjoHSt5bcVk0u,0gWQBwxwQRum86Rx4DaDHL,6VRhkROS2SZHGlp0pxndbJ,78XsyMWhm186OsTk9gghV6,02yaX0Reb1fQYIQdri4Ow8,4I27qMefHdVnH11HKtdfDL,3ETynfxvVCozp7I6qnqC17,30MdAjBfVTvBCS3vID38At,52Y9VvRfnKjdM0MvAM21cR,4BadVV8Jy1tuIj8C9xLJcH,0yz6PylgDjlUDkOZayhwOs,0yotOi6EFaBal5Gy7YVcuI,3CvIbxdih7XqVgtoRmYMAq,1ztnDy21iJOc5eH7wvbEeV,2BUDOrTS82npIINrWb1uXj,1ZFdXzBLNeAG3VotbwFo7r,1LVXkhEMVj4jUJKYMm3N77,6j7I2kiPvXXKjmmof3yybR,3AawFHorBGEDdM9ngI8ofa,0mlf3yOGzVnIVxjybDQL3h,76xfSfbw0byLqMHykJyPru,47UiEqtMQRGQ4vlCh7Lm4V,1gTa0kM9Xlfb2HS1KOzz4V,5b90v8q7CvaxNpoZbSd7VZ,2NX8NP9hZ0Cl97jrmgxJh6,5RD6iZpOFkVuT3SW4eAHuZ,1zk2MkgS2e

In [95]:
df = pd.concat([dubstep_df, house_df, dnb_df])
ftr_df = pd.concat([dubstep_ftrs, house_ftrs, dnb_ftrs])
timbres = np.concat([dub_timbres, house_timbres, dnb_timbres])

In [96]:
timbres_df = pd.DataFrame(timbres, columns=['t'+str(i) for i in range(timbres.shape[1])])
timbres_df = timbres_df.reset_index(drop=True)

In [99]:
ftr_df

,danceability,energy,loudness,acousticness,valence,time_signature
0,0.696,0.488,-7.508,0.19600,0.202,4
1,0.607,0.941,-3.807,0.03700,0.521,4
2,0.562,0.837,-5.007,0.07630,0.232,4
3,0.643,0.985,-1.848,0.01520,0.182,4
4,0.579,0.978,0.352,0.02050,0.580,4
...,...,...,...,...,...,...
98,0.353,0.987,-0.138,0.00471,0.211,4
99,0.745,0.981,-0.334,0.09270,0.682,4
100,0.444,0.945,-3.559,0.02720,0.147,5
101,0.744,0.906,-7.401,0.02380,0.530,4


In [97]:
timbres_df

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11
0,49.653779,27.938127,-43.586338,15.942574,-1.393539,-19.980069,6.306853,-4.741221,9.017387,4.143740,-31.084123,3.034637
1,50.403264,3.581097,-44.492222,-0.150778,54.399347,-38.457472,-5.054250,-30.289236,-5.190903,11.277528,-22.242847,-1.606222
2,54.773559,76.518596,9.966566,-1.101375,13.452265,-38.683632,10.288147,-2.375360,-9.164853,2.466228,-8.621640,-0.660809
3,56.957785,96.839608,-18.426367,-20.960253,13.899785,-34.915924,19.231051,7.022519,-7.193494,4.598253,-15.090430,5.493759
4,58.441559,134.660235,21.619206,-4.966029,17.869426,-40.899059,12.464059,10.404191,-8.720618,9.209897,-12.904309,-7.270132
...,...,...,...,...,...,...,...,...,...,...,...,...
730,58.383794,82.737118,-23.363059,-9.548676,16.239735,-54.271059,3.885971,1.625324,-20.077971,4.860647,-6.755588,12.668353
731,56.042847,90.586153,-22.505695,25.511023,40.913733,-36.123916,-24.693626,0.588313,-14.249221,18.907183,-19.215008,-3.944405
732,56.112255,89.388809,63.772447,6.923830,-0.135234,-16.326106,3.798872,6.293404,-33.730447,1.113957,-9.701213,-11.654468
733,49.899210,49.304657,12.743014,19.097629,35.385420,-36.387874,-6.188881,7.037259,-19.115748,11.153818,-10.096748,-2.839329


In [102]:
all_ftrs = pd.concat([ftr_df.reset_index(drop=True), timbres_df.reset_index(drop=True)], axis=1, ignore_index=True)
all_ftrs


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.696,0.488,-7.508,0.19600,0.202,4,49.653779,27.938127,-43.586338,15.942574,-1.393539,-19.980069,6.306853,-4.741221,9.017387,4.143740,-31.084123,3.034637
1,0.607,0.941,-3.807,0.03700,0.521,4,50.403264,3.581097,-44.492222,-0.150778,54.399347,-38.457472,-5.054250,-30.289236,-5.190903,11.277528,-22.242847,-1.606222
2,0.562,0.837,-5.007,0.07630,0.232,4,54.773559,76.518596,9.966566,-1.101375,13.452265,-38.683632,10.288147,-2.375360,-9.164853,2.466228,-8.621640,-0.660809
3,0.643,0.985,-1.848,0.01520,0.182,4,56.957785,96.839608,-18.426367,-20.960253,13.899785,-34.915924,19.231051,7.022519,-7.193494,4.598253,-15.090430,5.493759
4,0.579,0.978,0.352,0.02050,0.580,4,58.441559,134.660235,21.619206,-4.966029,17.869426,-40.899059,12.464059,10.404191,-8.720618,9.209897,-12.904309,-7.270132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,0.353,0.987,-0.138,0.00471,0.211,4,58.383794,82.737118,-23.363059,-9.548676,16.239735,-54.271059,3.885971,1.625324,-20.077971,4.860647,-6.755588,12.668353
731,0.745,0.981,-0.334,0.09270,0.682,4,56.042847,90.586153,-22.505695,25.511023,40.913733,-36.123916,-24.693626,0.588313,-14.249221,18.907183,-19.215008,-3.944405
732,0.444,0.945,-3.559,0.02720,0.147,5,56.112255,89.388809,63.772447,6.923830,-0.135234,-16.326106,3.798872,6.293404,-33.730447,1.113957,-9.701213,-11.654468
733,0.744,0.906,-7.401,0.02380,0.530,4,49.899210,49.304657,12.743014,19.097629,35.385420,-36.387874,-6.188881,7.037259,-19.115748,11.153818,-10.096748,-2.839329


In [67]:
len(timbres)

735

In [367]:
ftr_df

,danceability,energy,loudness,acousticness,valence,tempo,time_signature
0,0.696,0.488,-7.508,0.19600,0.202,95.064,4
1,0.607,0.941,-3.807,0.03700,0.521,75.401,4
2,0.562,0.837,-5.007,0.07630,0.232,145.059,4
3,0.643,0.985,-1.848,0.01520,0.182,140.029,4
4,0.579,0.978,0.352,0.02050,0.580,145.054,4
...,...,...,...,...,...,...,...
98,0.353,0.987,-0.138,0.00471,0.211,174.031,4
99,0.745,0.981,-0.334,0.09270,0.682,174.942,4
100,0.444,0.945,-3.559,0.02720,0.147,109.080,5
101,0.744,0.906,-7.401,0.02380,0.530,174.122,4


# EDA

In [280]:
for k in df:
    print(f'Number of unique values in {k} = {df[k].nunique()} ')


Number of unique values in danceability = 393 
Number of unique values in energy = 305 
Number of unique values in key = 12 
Number of unique values in loudness = 695 
Number of unique values in mode = 2 
Number of unique values in speechiness = 490 
Number of unique values in acousticness = 598 
Number of unique values in instrumentalness = 597 
Number of unique values in liveness = 445 
Number of unique values in valence = 513 
Number of unique values in tempo = 660 
Number of unique values in id = 727 
Number of unique values in duration_ms = 655 
Number of unique values in time_signature = 4 
Number of unique values in name = 714 


# Normalize

In [281]:
ftr_df = normalize(ftr_df)

# PCA

In [289]:
pca = PCA(n_components=3)
pca.fit(ftr_df)
reduced_ftrs = pca.transform(ftr_df)
reduced_ftrs

array([[-38.16654901,  -3.00489687,   0.49005245],
       [-57.76921982,   1.03415721,  -0.20729546],
       [ 11.86124188,  -1.25842687,  -0.31295182],
       ...,
       [-24.08941059,   0.75729204,  -0.2349908 ],
       [ 40.88279645,  -4.11430158,  -0.34404232],
       [-45.65665307,   2.18613581,  -0.17386663]])

# CUSTOM CLUSTERING

In [119]:
riddim = ['COUNTRY RIDDIM', 'KARATE_CHOP', '2KFO - Subtronics Remix', 'ID', 'Freaks', 'Get Fucked', 'Step Aside', 'Portals', 'Hit Em', 'Fire To The Roof', 'Thot Eliminated - VIP']

In [8]:
import numpy as np

In [120]:
# Choose songs to act as clusters #
# In final product would like to have a UI that makes this process easier #

riddim = ['COUNTRY RIDDIM', 'KARATE CHOP', '2KFO - Subtronics Remix', 'ID', 'Step Aside', 'Portals', 'Hit Em', 'Fire To The Roof', 'Thot Eliminated - VIP']
heavy = ['Fabrication', 'Mad Night', 'When I Look at You', 'Get By', 'Zodiac', 'In The Beginning', 'Super Hot', 'Out For Blood', 'Running']

def create_cluster_center(ftr_df, df, song_names):
    """
    Given song names make a cluster center
    """
    center = np.zeros_like(ftr_df[0])
    for name in song_names:
        # maybe think about hashing this to go faster #
        # think about how to handle duplicate names #
        index = df.index[df['name'] == name]
        row = ftr_df[index]
        center = row + center
    
    center /= len(song_names)
    print(center)
    return center[0]

def create_all_centers(ftr_df, df, cluster_songs):
    clusters = []
    for song_names in cluster_songs:
        clusters.append(create_cluster_center(ftr_df, df, song_names))
    
    return clusters


In [129]:
clusters = create_all_centers(dubstep_ftrs, dubstep_df, [riddim, heavy])

[[ 7.55333333e-01  9.26333333e-01  5.55555556e+00 -8.21222222e-01
   4.44444444e-01  1.17157778e-02  1.74134800e-01  3.41355556e-01
   3.78200000e-01  1.25222889e+02  4.00000000e+00]]
[[ 6.45888889e-01  7.91333333e-01  6.11111111e+00 -5.27144444e+00
   6.66666667e-01  1.55351111e-02  2.05869922e-01  2.65566667e-01
   2.23455556e-01  1.29984333e+02  3.66666667e+00]]


In [122]:
clusters

[array([-3.86402613, -0.17384568,  2.22922054, -0.13922496,  0.06786083,
         0.01821796]),
 array([ 0.88781127,  0.17789554, -2.24976742,  0.13611392, -0.25848014,
        -0.14018138])]

# KMEANS

In [127]:
def normalize(features):
    features -= np.mean(features, axis=0)
    features /= np.std(features, axis=0)
    return features

def cluster(ftrs, n_clusters, do_pca, num_pca_components, do_normalization):
    if do_normalization:
        ftrs = normalize(ftrs)
    if do_pca:
        pca = PCA(n_components=num_pca_components)
        pca.fit(ftrs)
        ftrs = pca.transform(ftrs)
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++').fit(ftrs)
    return kmeans

n_clusters = 3
do_pca = True
num_pca_components = 6
do_normalization = True
kmeans = cluster(all_ftrs, n_clusters, do_pca, num_pca_components, do_normalization)

In [128]:
cluster_names = [[] for i in range(n_clusters)]

for i in range(len(kmeans.labels_)):
    cluster_names[kmeans.labels_[i]].append(df.iloc[i]['name'])

# Evaluating effectiveness of clustering

In [129]:
for i, c in enumerate(cluster_names):
    counts = {"Dub": 0, "House": 0, "DNB": 0}
    songs = {"Dub": set(), "House": set(), "DNB": set()}
    for name in c:
        if name in dubstep_songs:
            counts['Dub'] += 1
            songs['Dub'].add(name)
        if name in house_songs:
            counts['House'] += 1
            songs['House'].add(name)
        if name in dnb_songs:
            counts['DNB'] += 1
            songs['DNB'].add(name)

    print(f"Cluster {i}")
    max_count = 0
    for k in counts:
        print(f"{k}: {counts[k]}")
        if counts[k] > max_count:
            genre = k
            max_count = counts[k]
    print("\n")

    print(f"Majority come from: {genre}")
    print("Songs not correpsonding to majority are:")
    for k in counts:
        if k != genre:
            for s in songs[k]:
                print(s)
    print("\n")

    # store songs that aren't corresponding to the majority


Cluster 0
Dub: 62
House: 163
DNB: 7


Majority come from: House
Songs not correpsonding to majority are:
EXPAND THE UNIVERSE
Control - Broadkast Remix
MOJITO
Explode
Hieroglyph - Of The Trees Remix
Never Let You Go Edit (Original by Sammy Virji)
Muscaria
Freaks
Zodiac
Bassface
Shape Shifter
Certified
Nicey
Spanish Moss
Griztronics
Malfunction - Original Mix
Ball So Hard - PIERCE Remix
Skanka
Babatunde
Eyes on Fire - Zeds Dead Remix
Claw
Starlight Lake (Outro)
Flip It
The Owl Song
Orgasmic
Jungle Juice
You & Me - Flume Remix
Monkey Funk
REKT
LFG
Supersonic (VIP)
Dear Weed Man (Ganja White Night Remix)
About Me Too
Painted
Raven
Shake Junt
Snickerz
Chemical Dreams
Black Ice
PSYCHEDELIC feat. Taylr Renee
Moon Slips Away
Take Flight Tech House VIP
Let's Get Weird
Hieroglyph VIP
Running
Fabrication
Gabagool
Next Level Style
Puzzle Box
Skanka - sumthin sumthin Remix
Durin's Tower
End of the World
GodLovesUgly - Remix
West Coast Wobble
Cockney Thug
Want It
Miss You
Noxious
Chicken Soup
Supers

In [62]:
cluster_names

[['Puzzle Box',
  'Griztronics',
  'Gummy Worm (feat. Boogie T)',
  'Open Your Mind',
  'Parisian Punch',
  'Satan Works Alone - Original Mix',
  'Tuba Demon',
  'Shimmer',
  'Dragons',
  'Vault - Subtronics Remix',
  'Chicken Soup',
  'Pray For Riddim VIP',
  'REKT',
  'Jungle Juice',
  'Spanish Moss',
  'Supa Fly',
  'Tuba Demon - HOL! Remix',
  'Hieroglyph - Of The Trees Remix',
  'FUNcKED - HE$H x Versa Remix',
  'Hieroglyph VIP',
  'Into Pieces VIP',
  'Take Flight Tech House VIP',
  'Miss You',
  'Psycho Cockz',
  'Painted',
  'Eidolon Hex',
  'SOMEWHERE',
  'Bad Boy',
  'Nicey',
  'Behemoth VIP',
  'Tonal Riddim',
  'The Owl Song',
  'Reminisce',
  'Malfunction - Original Mix',
  'Clockwork - Original Mix',
  'Womp Portal',
  'Headband',
  'Dear Weed Man (Ganja White Night Remix)',
  'ID - HOL! Remix',
  'LOVE ROCKET',
  'EXPAND THE UNIVERSE',
  'Resonate',
  'PSYCHEDELIC feat. Taylr Renee',
  'Gunslinger Feat. Pistol',
  'Space Bubblez',
  'Tommy Knocker',
  'Quickness',
  'Sta